In [13]:
import numpy as np
import os
import re
import sys

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from ipynb.fs.full.preprocessing import parse_poems
from HMM_helper import obs_map_reverser

In [14]:
# load data
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()

# get words as integers and get the word map
all_words = []
words, word_map = parse_poems(text)
for i in range(len(words)):
    for j in range(len(words[i])):
        all_words.append(words[i][j])
words = all_words

# get reverse map
int_to_words = obs_map_reverser(word_map)
        
n_words = len(words)
n_vocab = len(word_map)

In [5]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 7
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_words - seq_length, 1):
    seq_in = words[i:i + seq_length]
    seq_out = words[i + seq_length]
    dataX.append([word for word in seq_in])
    dataY.append(seq_out)
    
n_sequences = len(dataX)

In [6]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [7]:
# define the LSTM model 
# (single layer with 150 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(150, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
# load the network weights from best checkpoint
filename = "weights-improvement-word-20-6.3475.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [9]:
def sample(preds, temp=1.0):
    '''
    Helper function for sampling from softmax with different temperatures.
    
    Inputs:
    preds: output of softmax function
    temp: temperature to scale by
    '''
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

In [17]:
# set seed for emissions
seed = "shall i compare thee to a summers"
seed = seed.split(' ')
seed_to_int = [word_map[word] for word in seed]

# generate emissions
pattern = seed_to_int
temps = [0.1, 0.25, 0.75, 1.5]    
for temp in temps:    
    print('Temperature', temp)
    
    for j in range(14):
    
        # generate words
        for i in range(8):
            x = np.reshape(pattern, (1, len(pattern), 1))
            x = x / float(n_vocab)
            prediction = model.predict(x, verbose=0)[0]

            # sample according to temperature
            idx = sample(prediction, temp)

            #index = np.argmax(prediction[0])
            result = int_to_words[idx]
            pattern.append(idx)
            pattern = pattern[1:len(pattern)]

            # output result 
            sys.stdout.write(result + ' ')        
            if i == 7:
                sys.stdout.write('\n')
            sys.stdout.flush()

    print()

[84, 486, 745, 80, 28, 41, 276]
Temperature 0.1
i thy my and my thou and thy 
my thou and i and thy i thy 
and i and thy thy i and thy 
i i thy my and thou the the 
my and i i thy my thy my 
my thou i i and thy thy thy 
i and i my i my thy thou 
and and thy i and and thy my 
my and thy thy and thy thy thy 
my my thy my my thy thou thy 
thy i thy i thy my thy thy 
thou thy my to i thy thou and 
and in i thy my my and and 
thy thy i thy i thy thy and 

Temperature 0.25
love my and thy thy the in thy 
thou thy and i and my and i 
in my my the thy and i thy 
thy to i will thy then my my 
thy the and and i that my in 
thy and the i i i thy my 
in of love i to that to thou 
thy thy i and in my thy my 
my thy in i thy to thy self 
thy my and thy thy thy my and 
and thy and in thy i thy in 
thy and my to thy thou thy thou 
to as and is thy i and as 
thou thy thy and in not thy my 

Temperature 0.75
where long my self not thy self my 
the doth and to have and i idle 
of up i name and wilt me of